<a href="https://colab.research.google.com/github/mertege/Thesis_Experiments/blob/main/3_Spectrograms_CI4R_Datasets_77GHz_Multi_Class_LSTM_with_Attention_Window_Cropping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# LSTM and GRU based architecure is running in Gürbüz dataset with only 77GHz.
# Window Cropping is used.
# 3 LSTM and GRU blocks are used.
# Burada CI4R dataseti ile Multi Class Classification yapıldı. Sadece 77GHz train ve test edildi.
# 3 tane LSTM ve GRU var. Aralarında maxpooling1d(pool=4) var. Bidirectional(GRU) ve Bidirectional(LSTM) ile paralel olarak koşuyor ve ayrı ayrı attention yapılıyor. Sonrasında concat ile birleştiliyor. 
# Temporal pattern attention for multivariate time seriesforecasting makalesine göre yapıldı.
# 402x737 çözünürlüğünde gelen spectrogramın boyutunu "downscale" yöntemi ile küçültmek yerine time step'deki adımları 4'ün modu olacak şekilde 4 farklı datasete ayırdım. 
# Örneğin, (1,5,9,...),(2,6,10,...),(3,7,11,...) ve (4,8,12,...). 
# Böylece dataset'deki sample sayımı 4 katına çıkarmış oldum ve LSTM daha uzun süreli zamanı hafızasında tutabiliyor.
# Mean test accuracy is 0.876, max test accuracy is 0.886,
# Min test accuracy is 0.861,  std of test accuracy is 0.008.
# Time elapsed through all process: 440.21 sec
# Time elapsed during test time: 0.18, sec

In [2]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Conv1D,Activation, Dropout, Flatten, Dense, Reshape, BatchNormalization, ConvLSTM1D, Normalization, Input, Conv2D, MaxPooling1D, AveragePooling1D, MaxPooling2D, Concatenate, GRU, LSTM, TimeDistributed, Bidirectional, ReLU
from keras.regularizers import l2, l1
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 77 GHz
bending_77GHz                 = np.load('/content/drive/MyDrive/bending_77Ghz.npy')
bending_77GHz = np.float32(bending_77GHz)
label_bending_77GHz           = np.zeros((bending_77GHz.shape[0],1))
label_bending_77GHz[:]        = 1
crawling_77GHz                = np.load('/content/drive/MyDrive/crawling_77Ghz.npy')
crawling_77GHz = np.float32(crawling_77GHz)
label_crawling_77GHz          = np.zeros((crawling_77GHz.shape[0],1))
label_crawling_77GHz[:]       = 2
kneeling_77GHz                = np.load('/content/drive/MyDrive/kneeling_77Ghz.npy')
kneeling_77GHz = np.float32(kneeling_77GHz)
label_kneeling_77GHz          = np.zeros((kneeling_77GHz.shape[0],1))
label_kneeling_77GHz[:]       = 3
limping_with_RL_Stiff_77GHz   = np.load('/content/drive/MyDrive/limping_with_RL_Stiff_77Ghz.npy')
limping_with_RL_Stiff_77GHz = np.float32(limping_with_RL_Stiff_77GHz)
label_limping_with_RL_Stiff_77GHz           = np.zeros((limping_with_RL_Stiff_77GHz.shape[0],1))
label_limping_with_RL_Stiff_77GHz[:]        = 4
picking_up_an_object_77GHz    = np.load('/content/drive/MyDrive/picking_up_an_object_77Ghz.npy')
picking_up_an_object_77GHz = np.float32(picking_up_an_object_77GHz)
label_picking_up_an_object_77GHz          = np.zeros((picking_up_an_object_77GHz.shape[0],1))
label_picking_up_an_object_77GHz[:]       = 5
scissors_gait_77GHz           = np.load('/content/drive/MyDrive/scissors_gait_77Ghz.npy')
scissors_gait_77GHz = np.float32(scissors_gait_77GHz)
label_scissors_gait_77GHz          = np.zeros((scissors_gait_77GHz.shape[0],1))
label_scissors_gait_77GHz[:]       = 6
short_steps_77GHz             = np.load('/content/drive/MyDrive/short_steps_77Ghz.npy')
short_steps_77GHz = np.float32(short_steps_77GHz)
label_short_steps_77GHz          = np.zeros((short_steps_77GHz.shape[0],1))
label_short_steps_77GHz[:]       = 7
sitting_77GHz                 = np.load('/content/drive/MyDrive/sitting_77Ghz.npy')
sitting_77GHz = np.float32(sitting_77GHz)
label_sitting_77GHz          = np.zeros((sitting_77GHz.shape[0],1))
label_sitting_77GHz[:]       = 8
walking_away_from_Radar_77GHz = np.load('/content/drive/MyDrive/walking_away_from_Radar_77Ghz.npy')
walking_away_from_Radar_77GHz = np.float32(walking_away_from_Radar_77GHz)
label_walking_away_from_Radar_77GHz         = np.zeros((walking_away_from_Radar_77GHz.shape[0],1))
label_walking_away_from_Radar_77GHz[:]       = 9
Walking_on_both_toes_77GHz    = np.load('/content/drive/MyDrive/Walking_on_both_toes_77Ghz.npy')
Walking_on_both_toes_77GHz = np.float32(Walking_on_both_toes_77GHz)
label_Walking_on_both_toes_77GHz         = np.zeros((Walking_on_both_toes_77GHz.shape[0],1))
label_Walking_on_both_toes_77GHz[:]       = 10
Walking_towards_radar_77GHz   = np.load('/content/drive/MyDrive/Walking_towards_radar_77Ghz.npy')
Walking_towards_radar_77GHz = np.float32(Walking_towards_radar_77GHz)
label_Walking_towards_radar_77GHz          = np.zeros((Walking_towards_radar_77GHz.shape[0],1))
label_Walking_towards_radar_77GHz[:]       = 0


In [5]:
label_concat = label_bending_77GHz
del label_bending_77GHz
label_concat = np.concatenate((label_concat,label_crawling_77GHz),axis=0)  
del label_crawling_77GHz
label_concat = np.concatenate((label_concat,label_kneeling_77GHz),axis=0)  
del label_kneeling_77GHz
label_concat = np.concatenate((label_concat,label_limping_with_RL_Stiff_77GHz),axis=0)  
del label_limping_with_RL_Stiff_77GHz
label_concat = np.concatenate((label_concat,label_picking_up_an_object_77GHz),axis=0)  
del label_picking_up_an_object_77GHz
label_concat = np.concatenate((label_concat,label_scissors_gait_77GHz),axis=0)  
del label_scissors_gait_77GHz
label_concat = np.concatenate((label_concat,label_short_steps_77GHz),axis=0)  
del label_short_steps_77GHz
label_concat = np.concatenate((label_concat,label_sitting_77GHz),axis=0)  
del label_sitting_77GHz
label_concat = np.concatenate((label_concat,label_walking_away_from_Radar_77GHz),axis=0)  
del label_walking_away_from_Radar_77GHz
label_concat = np.concatenate((label_concat,label_Walking_on_both_toes_77GHz),axis=0)  
del label_Walking_on_both_toes_77GHz
label_concat = np.concatenate((label_concat,label_Walking_towards_radar_77GHz),axis=0)  
del label_Walking_towards_radar_77GHz


In [6]:
# 77 GHz
spectrogram_77GHz_concat = np.concatenate((bending_77GHz,crawling_77GHz),axis=0)
del bending_77GHz
del crawling_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,kneeling_77GHz               ),axis=0)
del kneeling_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,limping_with_RL_Stiff_77GHz  ),axis=0)
del limping_with_RL_Stiff_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,picking_up_an_object_77GHz   ),axis=0)
del picking_up_an_object_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,scissors_gait_77GHz          ),axis=0)
del scissors_gait_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,short_steps_77GHz            ),axis=0)
del short_steps_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,sitting_77GHz                ),axis=0)
del sitting_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,walking_away_from_Radar_77GHz),axis=0)
del walking_away_from_Radar_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_on_both_toes_77GHz   ),axis=0)
del Walking_on_both_toes_77GHz
spectrogram_77GHz_concat = np.concatenate((spectrogram_77GHz_concat,Walking_towards_radar_77GHz  ),axis=0)
del Walking_towards_radar_77GHz

In [7]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
# Concat range-doppler data
range_doppler_concat = spectrogram_77GHz_concat
range_doppler_concat_label = label_concat
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
# Concat range-doppler data
# spectrogram_concat = spectrogram_77GHz_concat
spectrogram_concat_label = onehot_encoded_labels
# Shuffle concat range doppler
# spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]
# Third Spectrogram
# third_spectrogram_concat = spectrogram_77GHz_concat
# third_spectrogram_concat_shuffle = third_spectrogram_concat[shuffle_indx,:,:,:]
del range_doppler_concat
del spectrogram_77GHz_concat

In [9]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mixup_augmentation(range_doppler_training_data, labels, repeat_of_mixup, alpha, beta):
    batch_size = range_doppler_training_data.shape[0]
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data.shape[2],range_doppler_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))

    # integer_training_labels = np.argmax(labels, axis=1)
    # np.where(integer_training_labels == 0)

    if repeat_of_mixup == 0:
      concat_images_range_doppler = range_doppler_training_data
      concat_label = labels      
    else:
      for ii in range(repeat_of_mixup):
        # shuffle train dataset
        shuffle_indx_1 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
        labels_shuffled_1 = labels[shuffle_indx_1,:]

        shuffle_indx_2 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
        labels_shuffled_2 = labels[shuffle_indx_2,:]

        # shuffle_indx_3 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        # range_doppler_training_data_shuffled_3 = range_doppler_training_data[shuffle_indx_3,:,:,:]
        # labels_shuffled_3 = labels[shuffle_indx_3,:]

        # shuffle_indx_4 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        # range_doppler_training_data_shuffled_4 = range_doppler_training_data[shuffle_indx_4,:,:,:]
        # labels_shuffled_4 = labels[shuffle_indx_4,:]

        # Sample lambda and reshape it to do the mixup
        # ll = np.random.beta(alpha, beta, (batch_size,1,1,1))
        gaussian_mean = 0.2
        gaussian_std = 0.02
        ll = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
        x_l = np.reshape(ll, (batch_size,1,1,1))
        y_l = np.reshape(ll, (batch_size,1))
      
        ll_2 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
        x_l_2 = np.reshape(ll_2, (batch_size,1,1,1))
        y_l_2 = np.reshape(ll_2, (batch_size,1))

        ll_3 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1,1,1))
        x_l_3 = np.reshape(ll_3, (batch_size,1,1,1))
        y_l_3 = np.reshape(ll_3, (batch_size,1))

        
        # Perform mixup on both images and labels by combining a pair of images/labels
        # images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * x_l_2 +\
        # range_doppler_training_data_shuffled_3 * x_l_3 + range_doppler_training_data_shuffled_4 * (1 - x_l - x_l_2 - x_l_3)
        # labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * y_l_2  + labels_shuffled_3 * y_l_3 + labels_shuffled_4 * (1 - y_l - y_l_2 - y_l_3)
        
        images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
        labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)

        concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
        concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

      concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
      concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images_range_doppler, concat_label)
def vertical_mixup_augmentation(range_doppler_training_data, labels, repeat_of_mixup, alpha, beta):
    batch_size = range_doppler_training_data.shape[0]
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),range_doppler_training_data.shape[1],range_doppler_training_data.shape[2],range_doppler_training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))

    # integer_training_labels = np.argmax(labels, axis=1)
    # np.where(integer_training_labels == 0)

    if repeat_of_mixup == 0:
      concat_images_range_doppler = range_doppler_training_data
      concat_label = labels      
    else:
      for ii in range(repeat_of_mixup):
        # shuffle train dataset
        shuffle_indx_1 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
        labels_shuffled_1 = labels[shuffle_indx_1,:]

        shuffle_indx_2 = random.sample(range(0, range_doppler_training_data.shape[0]), range_doppler_training_data.shape[0]) # split validation data
        range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
        labels_shuffled_2 = labels[shuffle_indx_2,:]

        images_mixup_range_doppler = np.zeros((range_doppler_training_data.shape))

        gaussian_mean = 0.15
        gaussian_std = 0.02
        ll_1 = np.random.normal(gaussian_mean, gaussian_std, (batch_size,1))
        vertical_resolution = range_doppler_training_data_shuffled_1.shape[2]
        index_number_1 = int(vertical_resolution*ll_1[0])
        index_number_2 = vertical_resolution - index_number_1
        shuffle_indx_1 = random.sample(range(0, vertical_resolution), index_number_1) # split validation data
        shuffle_indx_2 = np.delete(range(0, vertical_resolution), shuffle_indx_1) # split training data

        images_mixup_range_doppler[:,:,shuffle_indx_1,0] = range_doppler_training_data_shuffled_1[:,:,shuffle_indx_1,0]
        images_mixup_range_doppler[:,:,shuffle_indx_2,0] = range_doppler_training_data_shuffled_2[:,:,shuffle_indx_2,0]
        
        # images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)

        labels_mixup = labels_shuffled_1 * ll_1 + labels_shuffled_2 * (1 - ll_1)

        concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
        concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

      concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
      concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images_range_doppler, concat_label)
def split_and_augmentation_of_training(range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  # to split train and validation with same distribution
  alpha = 0.1
  beta = 5
  dummy_label = np.zeros((range_doppler_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(range_doppler_concat_shuffle_train,dummy_label):   
    randomlist_for_validation_indx
  # size_of_validation = int(0.2*range_doppler_concat_shuffle_train.shape[0])
  # # Split validation
  # randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  # randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  (range_doppler_augmented_image,spectrograms_label)=\
   mixup_augmentation(range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha, beta)

  # (range_doppler_augmented_image,spectrograms_label)=\
  #  vertical_mixup_augmentation(range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha, beta)

  return (range_doppler_augmented_image,spectrograms_label,\
     range_doppler_validation_data, spectrogram_validation_labels)



def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [10]:
def time_steps_divided_4(range_doppler_concat_shuffle, range_doppler_concat_label_shuffle):
  range_doppler_concat_shuffle_sub1 = np.zeros((range_doppler_concat_shuffle.shape[0],range_doppler_concat_shuffle.shape[1],184,1))
  range_doppler_concat_shuffle_sub2 = np.zeros((range_doppler_concat_shuffle.shape[0],range_doppler_concat_shuffle.shape[1],184,1))
  range_doppler_concat_shuffle_sub3 = np.zeros((range_doppler_concat_shuffle.shape[0],range_doppler_concat_shuffle.shape[1],184,1))
  range_doppler_concat_shuffle_sub4 = np.zeros((range_doppler_concat_shuffle.shape[0],range_doppler_concat_shuffle.shape[1],184,1))
  for ii in range(range_doppler_concat_shuffle.shape[0]):
    for jj in range(184):
      range_doppler_concat_shuffle_sub1[ii,:,jj,:] = range_doppler_concat_shuffle[ii,:,jj*4,:]
      range_doppler_concat_shuffle_sub2[ii,:,jj,:] = range_doppler_concat_shuffle[ii,:,jj*4+1,:]
      range_doppler_concat_shuffle_sub3[ii,:,jj,:] = range_doppler_concat_shuffle[ii,:,jj*4+2,:]   
      range_doppler_concat_shuffle_sub4[ii,:,jj,:] = range_doppler_concat_shuffle[ii,:,jj*4+3,:]
  range_doppler_concat_shuffle = range_doppler_concat_shuffle_sub1
  range_doppler_concat_shuffle = np.concatenate((range_doppler_concat_shuffle,range_doppler_concat_shuffle_sub2),axis=0) 
  range_doppler_concat_shuffle = np.concatenate((range_doppler_concat_shuffle,range_doppler_concat_shuffle_sub3),axis=0) 
  range_doppler_concat_shuffle = np.concatenate((range_doppler_concat_shuffle,range_doppler_concat_shuffle_sub4),axis=0)
  range_doppler_concat_label_shuffle = np.concatenate((range_doppler_concat_label_shuffle,range_doppler_concat_label_shuffle),axis=0) 
  range_doppler_concat_label_shuffle = np.concatenate((range_doppler_concat_label_shuffle,range_doppler_concat_label_shuffle),axis=0)  
  return range_doppler_concat_shuffle, range_doppler_concat_label_shuffle

In [11]:
def time_steps_divided_2(range_doppler_concat_shuffle, range_doppler_concat_label_shuffle):
  range_doppler_concat_shuffle_sub1 = np.zeros((range_doppler_concat_shuffle.shape[0],range_doppler_concat_shuffle.shape[1],368,1))
  range_doppler_concat_shuffle_sub2 = np.zeros((range_doppler_concat_shuffle.shape[0],range_doppler_concat_shuffle.shape[1],368,1))
  for ii in range(range_doppler_concat_shuffle.shape[0]):
    for jj in range(368):
      range_doppler_concat_shuffle_sub1[ii,:,jj,:] = range_doppler_concat_shuffle[ii,:,jj*2,:]
      range_doppler_concat_shuffle_sub2[ii,:,jj,:] = range_doppler_concat_shuffle[ii,:,jj*2+1,:]

  range_doppler_concat_shuffle = range_doppler_concat_shuffle_sub1
  range_doppler_concat_shuffle = np.concatenate((range_doppler_concat_shuffle,range_doppler_concat_shuffle_sub2),axis=0) 
  range_doppler_concat_label_shuffle = np.concatenate((range_doppler_concat_label_shuffle,range_doppler_concat_label_shuffle),axis=0) 
  return range_doppler_concat_shuffle, range_doppler_concat_label_shuffle

In [12]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)

In [13]:
from keras.layers import *
from keras.models import *
from keras import backend as K

class attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        # a = K.sigmoid(e)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [14]:
# range_doppler_concat_shuffle = (range_doppler_concat_shuffle)/255

In [15]:
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
# Parameters
epoch_number = 100
batch_size = 32
repeat_of_mixup = 0
number_of_repeat = 5
unit_number_of_lstm = 128
conv1d_filters = 256
conv1d_kernel_size = 25
# Regularizers
lstm_dropout_rate = 0.15
recurrent_dropout_rate = 0.0
recurrent_regularizer_value = l2(0.002)
recurrent_regularizer_conv1d = l2(0.01)
n_classes = 128
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  for train, test in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()

    randomlist_for_test_indx = test
    randomlist_for_train_indx = train
    # test data
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    #train data
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    # ---------------- Split labels to equal them during augmentation for Validation ----------------
    (range_doppler_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_range_doppler, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(range_doppler_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)
      
    range_doppler_augmented_image, spectrogram_concat_label_shuffle_concat = time_steps_divided_4(range_doppler_augmented_image, spectrogram_concat_label_shuffle_concat)
    validation_range_doppler, spectrogram_validation_labels = time_steps_divided_4(validation_range_doppler, spectrogram_validation_labels)
    range_doppler_concat_shuffle_test, spectrogram_concat_label_shuffle_test = time_steps_divided_4(range_doppler_concat_shuffle_test, spectrogram_concat_label_shuffle_test)

    # range_doppler_augmented_image, spectrogram_concat_label_shuffle_concat = time_steps_divided_2(range_doppler_augmented_image, spectrogram_concat_label_shuffle_concat)
    # validation_range_doppler, spectrogram_validation_labels = time_steps_divided_2(validation_range_doppler, spectrogram_validation_labels)
    # range_doppler_concat_shuffle_test, spectrogram_concat_label_shuffle_test = time_steps_divided_2(range_doppler_concat_shuffle_test, spectrogram_concat_label_shuffle_test)

    range_doppler_augmented_image = np.transpose(range_doppler_augmented_image, axes = (0,2,1,3)) 
    validation_range_doppler = np.transpose(validation_range_doppler, axes = (0,2,1,3)) 
    range_doppler_concat_shuffle_test = np.transpose(range_doppler_concat_shuffle_test, axes = (0,2,1,3))



    def lstm_encoder_network_1(input_shape):
      input = Input(shape=input_shape)
      x = Bidirectional(GRU(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                             recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                    
      x = AveragePooling1D(pool_size=2, strides=None)(x)
      x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
                             recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
      x = AveragePooling1D(pool_size=2, strides=None)(x)      
      x = Bidirectional(GRU(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
                             recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
      x = AveragePooling1D(pool_size=2, strides=None)(x)                          
      x = tf.transpose(x, perm=[0,2,1])
      x = attention(return_sequences=True)(x)
      x = tf.transpose(x, perm=[0,2,1])        
      x = GlobalAveragePooling1D()(x)
      return Model(input, x)


    def lstm_encoder_network_2(input_shape):
      input = Input(shape=input_shape)
      x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = lstm_dropout_rate,
                             recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(input)                            
      x = AveragePooling1D(pool_size=2, strides=None)(x)
      x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
                             recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
      x = AveragePooling1D(pool_size=2, strides=None)(x)      
      x = Bidirectional(LSTM(int(unit_number_of_lstm), return_sequences=True, dropout = int(lstm_dropout_rate),
                             recurrent_dropout= recurrent_dropout_rate, kernel_regularizer=recurrent_regularizer_value))(x) 
      x = AveragePooling1D(pool_size=2, strides=None)(x)  
      x = tf.transpose(x, perm=[0,2,1])
      x = attention(return_sequences=True)(x)
      x = tf.transpose(x, perm=[0,2,1])        
      x = GlobalAveragePooling1D()(x)
      return Model(input, x)
  
    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = BatchNormalization()(input)
      # x = Dense(512)(x)
      # x = BatchNormalization()(x)
      # x = Activation('LeakyReLU')(x)
      # x = Dropout(0.25)(x)
      x = Dense(64)(x)
      x = BatchNormalization()(x)
      x = Activation('LeakyReLU')(x)
      x = Dropout(0.15)(x)
      x = Dense(11, activation="softmax")(x)
      return Model(input, x)

    input_shape = range_doppler_augmented_image.shape[1:3]

    base_network_lstm_1 = lstm_encoder_network_1(input_shape)
    lstm_input_1  = Input(shape=input_shape)
    processed_lstm_1  = base_network_lstm_1(lstm_input_1)

    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    processed_lstm_2  = base_network_lstm_2(lstm_input_1)

    concat_layer = Concatenate()([processed_lstm_1, processed_lstm_2])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1]))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[lstm_input_1], outputs=[out])

    print(model.summary()) 

    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=35, verbose=0,restore_best_weights=True, mode='min')
    t = time.time()
    history = model.fit((range_doppler_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    verbose = 0,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler) , (spectrogram_validation_labels)))
    tf.keras.models.load_model
    elapsed = time.time() - t
    predictions = model.predict(range_doppler_concat_shuffle_test)
    length_of_splitted_label = int(predictions.shape[0]/4)
    # majority_voting = (predictions[:length_of_splitted_label,:] + predictions[length_of_splitted_label:,:])/2
    majority_voting = (predictions[:length_of_splitted_label,:] + predictions[length_of_splitted_label:length_of_splitted_label*2,:]\
                       + predictions[length_of_splitted_label*2:length_of_splitted_label*3,:] + predictions[length_of_splitted_label*3::,:])/4
    correct = 0
    total = 0
    for i in range(len(majority_voting)):
        act_label = np.argmax(spectrogram_concat_label_shuffle_test[i]) # act_label = 1 (index)
        pred_label = np.argmax(majority_voting[i]) # pred_label = 1 (index)
        if(act_label == pred_label):
            correct += 1
        total += 1
    test_accuracy = (correct/total)
    # test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test],\
    #                                            [spectrogram_concat_label_shuffle_test],
    #               batch_size=batch_size)

    test_accuracy_per_fold.append(test_accuracy)
    print(test_accuracy_per_fold)
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  print(test_accuracy_per_run)

print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')

print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 184, 402)]   0           []                               
                                                                                                  
 model (Functional)             (None, 256)          1001751     ['input_2[0][0]']                
                                                                                                  
 model_1 (Functional)           (None, 256)          1332503     ['input_2[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 512)          0           ['model[0][0]',                  
                                                                  'model_1[0][0]']          

In [17]:
t_test = time.time()
y_test_predicted = model.predict((range_doppler_concat_shuffle_test[0:1,:,:,:]))
test_elapsed = time.time() - t_test 
print(f'Time elapsed during test time: {"{:.2f}".format(test_elapsed)}, sec')

Time elapsed during test time: 0.18, sec


In [18]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},')
print(f'Min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)} sec')
print(f'Time elapsed during test time: {"{:.2f}".format(test_elapsed)}, sec')

Mean test accuracy is 0.876, max test accuracy is 0.886,
Min test accuracy is 0.861,  std of test accuracy is 0.008.
Time elapsed through all process: 440.21 sec
Time elapsed during test time: 0.18, sec
